## Import Module

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Environment Class

In [ ]:
class Environment:
    
    # constructor
    def __init__(self, company1, company2, price_col, training_dataset_ratio=0.8, nrm=1, one_episode_num_step=30):
        
        self.company1 = company1
        self.stock_price1 = None
        self.stock_price1_train = None
        self.stock_price1_test = None
        
        self.company2 = company2
        self.stock_price2 = None
        self.stock_price2_train = None
        self.stock_price2_test = None
        
        self.price_col = price_col
        self.stock_price_length = None
        
        self.training_dataset_ratio = training_dataset_ratio
        self.nrm = nrm
        
        self.stock_price_final = []
        self.local_current_step = None
        self.global_current_step = None
        self.purpose = None
        self.old_prupoese = None
        self.one_episode_num_step = one_episode_num_step
        
        # read two company's stock price
        self.load_data()
        
        # check if two stock price have same date-index
        
        
        # extract close price from two stock price and convert to numpy array
        self.extract_close_price()
        
        # split two stock close price into training set and testing set
        self.split_dataset()
        
    
    # read two company's stock price
    def load_data(self):
        
        try:
            self.stock_price1 = pd.read_csv(self.company1 + ".csv")
        except:
            raise Exception("Cannot load {}".format(self.company1 + ".csv"))
            
        try:
            self.stock_price2 = pd.read_csv(self.company2 + ".csv")
        except:
            raise Exception("Cannot load {}".format(self.company2 + ".csv"))
            
    
    # extract close price from two stock price and convert to numpy array
    def extract_close_price(self):
        
        try:
            self.stock_price1 = self.stock_price1[self.price_col]
            self.stock_price2 = self.stock_price2[self.price_col]
        except:
            raise Exception("Cannot extract stock price column: {}.".format(self.price_col))
            
        
        self.stock_price1 = self.stock_price1.values
        self.stock_price2 = self.stock_price2.values
        self.stock_price_length = len(self.stock_price1)
        
        
    # split two stock close price into training set and testing set
    def split_dataset(self):
        
        index = round(self.stock_price_length * self.training_dataset_ratio)
        
        self.stock_price1_train = self.stock_price1[:index]
        self.stock_price1_test = self.stock_price1[index:]
        
        self.stock_price2_train = self.stock_price2[:index]
        self.stock_price2_test = self.stock_price2[index:]
        
        print("===============Environment Info===============")
        print("Stock1: {}".format(self.company1))
        print("Stock2: {}".format(self.company2))
        print("Price Column: {}".format(self.price_col))
        print("Ngative Return Multiplier: {}".format(self.nrm))
        print("Total number of day for training: {}".format(str(len(self.stock_price1_train))))
        print("Total number of day for testing: {}".format(str(len(self.stock_price1_test))))
        print("==============================================")
        
    
    # reset environment: must specify purpose for training or tetsing
    def reset(self, purpose):
        
        self.purpose = purpose
        
        if self.purpose != self.old_prupoese:
            self.prepare_final_data()
            self.old_prupoese = self.purpose
            self.global_current_step = 0
        
        self.global_current_step += 1
        self.local_current_step = 0
        
        
        if self.global_current_step == len(self.stock_price_final)-self.one_episode_num_step+1:
            self.global_current_step = 0
            
        return self.stock_price_final[self.global_current_step]
            
            
    # prepare train data
    def prepare_final_data(self):
        
        # spread of two stock
        if self.purpose == "train":
            spread = self.stock_price1_train - self.stock_price2_train
        else:
            spread = self.stock_price1_test - self.stock_price2_test
        
        
        for idx, value in enumerate(spread[:-1]):
            
            one_step = np.empty(shape=(10))
            one_step_idx = 0
            
            # current spread
            current_spread = value
            one_step[one_step_idx] = current_spread
            one_step_idx += 1
            
            # daily return of spread
            daily_return_spread = spread[idx+1] - current_spread
            one_step[one_step_idx] = daily_return_spread
            one_step_idx += 1
            
            # spread mean during past 15 days
            if idx != 0:
                temp_idx = 0 if (idx-15<0) else (idx-15)
                spread_mean_15_days = np.mean(spread[temp_idx:idx])
            else:
                spread_mean_15_days = value
            
            one_step[one_step_idx] = spread_mean_15_days
            one_step_idx += 1
            
            # curren spread / spread mean during past 15 days
            one_step[one_step_idx] = current_spread / spread_mean_15_days
            one_step_idx += 1
            
            # spread mean during past 10 days
            if idx != 0:
                temp_idx = 0 if (idx-10<0) else (idx-10)
                spread_mean_10_days = np.mean(spread[temp_idx:idx])
            else:
                spread_mean_10_days = value
            one_step[one_step_idx] = spread_mean_10_days
            one_step_idx += 1
            
            # curren spread / spread mean during past 10 days
            one_step[one_step_idx] = current_spread / spread_mean_10_days
            one_step_idx += 1
            
            # spread mean during past 7 days
            if idx != 0:
                temp_idx = 0 if (idx-7<0) else (idx-7)
                spread_mean_7_days = np.mean(spread[temp_idx:idx])
            else:
                spread_mean_7_days = value
            one_step[one_step_idx] = spread_mean_7_days
            one_step_idx += 1
            
            # curren spread / spread mean during past 7 days
            one_step[one_step_idx] = current_spread / spread_mean_7_days
            one_step_idx += 1
            
            # spread mean during past 5 days
            if idx != 0:
                temp_idx = 0 if (idx-5<0) else (idx-5)
                spread_mean_5_days = np.mean(spread[temp_idx:idx])
            else:
                spread_mean_5_days = value
            one_step[one_step_idx] = spread_mean_5_days
            one_step_idx += 1
            
            # curren spread / spread mean during past 5 days
            one_step[one_step_idx] = current_spread / spread_mean_5_days
            one_step_idx += 1
            
            self.stock_price_final.append(one_step)
            
    
    # go next step: must provide action
    def step(self, action):
        
        # new state
        self.local_current_step += 1
        step_idx = self.global_current_step + self.local_current_step
        new_state = self.stock_price_final[step_idx]
        
        # reward
        reward = action * self.stock_price_final[step_idx-1][1]
        if self.purpose == "train":
            reward = reward * self.nrm if reward < 0 else reward
            
        # is done
        done = True if((self.local_current_step == self.one_episode_num_step-1) or (self.global_current_step+self.local_current_step==len(self.stock_price_final)-1)) else False
        
        return new_state, reward, done

## Agent Class

In [ ]:
class Agent:
    
    # constructor
    def __init__(self, 
                 state_dim,
                 action_dim,
                 learning_rate,
                 exploration_rate, 
                 exploration_decay,
                 exploration_min, 
                 replay_buffer_size,
                 batch_size, 
                 q_value_discounter
                ):
        
        # state dimension
        self.state_dim = state_dim
        
        # action dimension
        self.action_dim = action_dim
        
        # learning rate
        self.learning_rate = learning_rate
        
        # exploration rate, decay and min for agent
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        
        # replay buffer size, batch size for agent
        self.replay_buffer_size = replay_buffer_size
        self.batch_size = batch_size
        
        # replay_buffer will store: state1, action1, reward1, state2, done1
        self.replay_buffer = np.empty((self.replay_buffer_size, 10+1+1+10+1))
        self.replay_buffer_counter = 0
        
        # discounter of max q value among actions 
        self.q_value_discounter = q_value_discounter
        
        # build a critic
        self.critic = self.build_nn()
        self.critic.summary()
        
    
    # build a neural network as critic
    def build_nn(self):
        
        inputs = keras.Input(shape=(self.state_dim, ), name="InputLayer")
        hidden1 = keras.layers.Dense(20, activation="relu", name="HiddenLayer1")(inputs)
        hidden2 = keras.layers.Dense(10, activation="relu", name="HiddenLayer2")(hidden1)
        outputs = keras.layers.Dense(3, activation="linear", name="OutputLayer")(hidden2)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        
        return model
    

    # smaple action given a state
    def sample_action(self, state):
        
        explore = np.random.uniform(low=0.0, high=1.0)
        
        # explore
        if explore < self.exploration_rate:
            action_index = np.random.choice(self.action_dim)
        
        else:
            state = np.reshape(state, (1, self.state_dim))
            action = self.critic.predict(state)[0]
            action_index = np.argmax(action)
        
        # action should be: 0, 1, -1
        action_index -= 1
        
        return action_index
        
        
    # store experience into replay buffer
    def store_experience(self, state1, action1, reward1, state2, done1):
        
        experience = np.empty((10+1+1+10+1))
        experience[0:10] = state1[:]
        experience[10] = action1
        experience[11] = reward1
        experience[12:22] = state2[:]
        experience[22] = int(done1)
        
        self.replay_buffer[self.replay_buffer_counter % self.replay_buffer_size][:] = experience
        self.replay_buffer_counter += 1
        
        
    # train critic with experience from replay buffer
    def train(self):
        
        if self.replay_buffer_counter < self.batch_size:
            return
        
        # sample batch of experience from replay buffer
        low_idx = 0
        high_idx = min(self.replay_buffer_counter, self.replay_buffer_size)
        mask = np.random.randint(low=low_idx, high=high_idx, size=self.batch_size)
        batch_experience = self.replay_buffer[mask][:]
        
        # each experience: state1, action1, reward1, state2, done1
        for experience in batch_experience:
            
            state1 = experience[0:10]
            action1 = experience[10]
            reward1 = experience[11]
            state2 = experience[12:22]
            done1 = experience[22]
            
            # if state2 is final state in its episode
            if done1 == 1:
                target_reward = reward1
            
            # if not
            else:
                state = np.reshape(state2, (1, self.state_dim))
                max_action_value = np.max(self.critic.predict(state)[0])
                target_reward = reward1 + self.q_value_discounter * max_action_value
            
            state = np.reshape(state1, (1, self.state_dim))
            action_value = self.critic.predict(state)
            action_value[0][int(action1+1)] = target_reward
            
            self.critic.fit(x=state, y=action_value, verbose=0)
                
        
        # decrease exploraction rate
        self.exploration_rate = max(self.exploration_min, self.exploration_rate*self.exploration_decay)

## PairTrading Class

In [ ]:
class PairTrading:
    
    # constructor
    def __init__(self):
        
        # state dimension 
        state_dim = 10
        
        # action dimension
        action_dim = 3
        
        # learning rate for agent
        learning_rate = 0.001
        
        # exploration rate, decay and min for agent
        exploration_rate = 1
        exploration_decay = 0.995
        exploration_min = 0.01
        
        # replay buffer size, batch size for agent
        replay_buffer_size = 500
        batch_size = 50
        
        # discounter of max q value among actions 
        q_value_discounter = 0.95
        
        # build agent
        self.agent = Agent(state_dim=state_dim,
                           action_dim=action_dim,
                           learning_rate=learning_rate,
                           exploration_rate=exploration_rate,
                           exploration_decay=exploration_decay,
                           exploration_min=exploration_min,
                           replay_buffer_size=replay_buffer_size,
                           batch_size=batch_size,
                           q_value_discounter=q_value_discounter
                           )
        
        # build environment
        self.env = Environment(company1="AAPL",
                               company2="GOOG", 
                               price_col="Close",
                               training_dataset_ratio=0.99,
                               nrm=500,
                               one_episode_num_step=40)
        
        # total episode to play
        self.total_episode = 2000
        
        # store every episode's total reward
        self.episode_total_reward = []
        
        
    # start to play pair trading
    def start(self):
        
        for episode in range(self.total_episode):
            
            # total rewardin each episode
            total_reward = 0
            
            # a flag to indicate if the episode ends
            done = False
            
            # get environment's initial state
            state1 = self.env.reset(purpose="train")
            
            while done is False:
                
                # agent samples an action given state
                action1 = self.agent.sample_action(state1)
                
                # environemnt enters next state given action
                state2, reward1, done1 = self.env.step(action1)
                
                # store this step (experience)
                self.agent.store_experience(state1, action1, reward1, state2, done1)
                
                # update local variable
                state1 = state2
                done = done1
                total_reward += reward1
                
            # when this episode ends...
            print("#%.3d Episode's Reward: %.4d" %(episode, total_reward))
            self.episode_total_reward.append(total_reward)
            self.agent.train()
            
    def plot_result(self):
        
        x = list(range(0, self.total_episode))
        y = self.episode_total_reward
        plt.plot(x, y, color="blue", label="nrm={}".format(str(self.env.nrm)))
        plt.xlabel("Episode")
        plt.ylabel("Total Reward")
        plt.title("DQN Agent's Performance on Pair Trading")
        plt.legend(loc="upper left")
        plt.savefig("{}.png".format(str(self.env.nrm)))
        plt.show()

## Main

In [ ]:
pair_trading_game = PairTrading()
pair_trading_game.start()
pair_trading_game.plot_result()